#  [INSERT TITLE]
- Date:  
- Author:  Bill Louer
- Datasource:  

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/9/96/Generic-person.svg/1024px-Generic-person.svg.png' alt="Generic Image" width="400" height="100" >
[GENERIC IMAGE]

##  Table of Contents
1.  [Project Overview](#section-1) 
2.  [Data Input](#section-2)
3.  [Data Cleaning](#section-3)
4.  [Exploratory Data Analysis](#section-4)
5.  [Feature Engineering](#section-5)
6.  [Modeling and Evaluation](#section-6)
7.  [Findings and Conclusions](#section-7)

##  1. Project Overview <a class="anchor" id="section-1"></a>

###  Objectives:  
-  

#### Dataset source:  https://www.kaggle.com/datasets/aagmandeep/combined-cycle-power-plant-dataset-and-prediction
##### Data Contents:
- The dataset comprises 

In [1]:
#  import libaries


In [2]:
# Set pandas display precision for floats
# pd.set_option('display.precision', 3)  # Set to 2 decimal places

## 2. Data Input <a class="anchor" id="section-2"></a>

In [3]:
#  Set up miscellaneous function so I can print dataframe tables for the report or presentation.
# This function requires work.  I need to format data columns with the appropriate number of significant digits before printing.
def print_table(df, sheetname):
    filepath = '../reports/results.xlsx'
    if os.path.exists(filepath):
        with pd.ExcelWriter(filepath, mode='a', engine='openpyxl', if_sheet_exists='new') as writer:
            # Write DataFrame to a new sheet
            df.to_excel(writer, sheet_name=sheetname, index=False,  float_format="%.3f")
    else:
        with pd.ExcelWriter(filepath, mode='w', engine='openpyxl',) as writer:
            # Write DataFrame to a new sheet
            df.to_excel(writer, sheet_name=sheetname, index=False, float_format="%.3f")


## 3.  Data Cleaning <a class="anchor" id="section-3"></a>

In [4]:
#  check for NA values


In [5]:
# Produce and save boxplots to show the distribution of each variables data.  Show where they may be outliers.


In [6]:
# Produce and save boxplots to show the distribution of each variables data.  Show where they may be outliers.


In [7]:
#  Look at outliers using a boxplot


In [8]:
#  Create a histogram of each variable


In [9]:
#  Identify outliers by z_scores


## 4. Exploratory Data Analysis <a class="anchor" id="section-4"></a>

## 5. Feature Engineering <a class="anchor" id="section-5"></a>

#####  Output is dependent most on dry bulb, wet bulb and steam turbine backpressure (cm_Hg).  
#####  Lets check correlation and p_values for all features.

In [12]:
# columns = list(df.columns)[:-1]
# y_col = 'output_MW'
# x_vars, y_vars, corr_coefs, p_vals = [], [], [], []
# for col in columns:
#     x_vars.append(col)
#     y_vars.append('output_MW')
#     coef, p_val = stats.pearsonr(df[col], df[y_col])
#     corr_coefs.append(coef)
#     p_vals.append(p_val)
# df_corr_p_val = pd.DataFrame({'x_variable': x_vars, 'y_variable':y_vars, 'correlation_coef': corr_coefs, 'p_values': p_vals})
# print_table(df_corr_p_val, 'correlation_p_vals')
# df_corr_p_val

In [13]:
Display heatmap of correlation between variables.
# sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
# plt.title('Heat Map of Correlation Coefficients')
# plt.savefig('../reports/figures/corr_heatmap.png', dpi=300, bbox_inches='tight')
# plt.show()

SyntaxError: invalid syntax (3119082695.py, line 1)

#### Save the cleaned and featured data to a new csv file.

In [ ]:
#  filepath = '../data/interim/interim_data.csv'
# df.to_csv(filepath, index=False )

####  Scale the feature variables in the data using standard scalar.

In [ ]:
#  Create training and testing sets and then scale the feature variables.  This prevents data leakage (test data leaking into training the model.)


## 6. Modeling and Evaluation <a class="anchor" id="section-6"></a>

In [ ]:
#  plot a kde plot of predicted MW and the actual MW
def plot_test_prediction_distribution(y_test, y_predict, model_type, mse, mae, r2):
    fig, axs = plt.subplots(1,2, figsize=(12,6))
    sns.set(style="whitegrid")
    sns.kdeplot(ax=axs[0], x=y_test, color="r", label="Actual Values")
    sns.kdeplot(ax=axs[0], x=y_predict, color="b", label="Predicted Value")
    
    axs[0].set_title(f'Distribution Plot ({model_type})\nOut-of-Sample Predicted vs. Actual Test Data',  fontsize=10, color='black')
    axs[0].annotate(f'Model Fit:\nMSE:  {mse:.04f} \nMAE: {mae:.01f}\nR2:  {r2:.03f}', xy=(0.65, 0.85), xycoords='axes fraction',
                       fontsize=10, color='black', ha='left', va='top')
    axs[0].legend()
    
    axs[1].scatter(mw_predict, y_test, alpha = 0.4)
    axs[1].set_title(f'Predicted vs. Actual\n({model_type})')
    axs[1].set_xlabel('Model Predicted MW Output')
    axs[1].set_ylabel('Actual MW Output')
    plt.savefig(f'../reports/figures/{model_type}_kde.png', dpi=300, bbox_inches='tight')
    plt.show()

In [ ]:
#  Show a distribution of the percent error of the prediction and the confidence interval on the kde plot.
def plot_error_distribution(y_test, y_predict, model_type, lwr_pct=2.5, uppr_pct=97.5):
    '''Plot the error distribution given the test data set and predicted data set'''
    variance_pct = 100*(y_test - y_predict)/y_test
    plt.figure(figsize=(10,5))
    sns.kdeplot(x=variance_pct, fill=False, color="blue", label="Variance Distribution")
    plt.title(f'Distribution of Percent Error\nModel Type:  {model_type}')
    plt.xlabel('Percent Error')
    
    # Compute the KDE values and x-axis values
    kde = stats.gaussian_kde(variance_pct)
    x_vals = np.linspace(min(variance_pct), max(variance_pct), 1000)
    kde_vals = kde(x_vals)
    
    # Calculate the percentiles for the middle 90% (5th and 95th percentiles)
    lower_bound = np.percentile(variance_pct, lwr_pct)
    upper_bound = np.percentile(variance_pct, uppr_pct)
    
    # Plot the KDE
    plt.plot(x_vals, kde_vals, color='blue')
    
    # Shade the middle 90%
    plt.fill_between(x_vals, kde_vals, where=(x_vals >= lower_bound) & (x_vals <= upper_bound), 
                     color='blue', alpha=0.3)
    
    # Highlight the percentile values on the x-axis
    plt.axvline(lower_bound, color='red', linestyle='--', label=f'2.5th percentile: {lower_bound:.01f}% Error')
    plt.axvline(upper_bound, color='red', linestyle='--', label=f'97.5th percentile: {upper_bound:.01f}% Error')
    
    plt.legend()
    plt.savefig(f'../reports/figures/{model_type}_error_dist.png', dpi=300, bbox_inches='tight')
    plt.show()

19


In [ ]:
# #  Document the model results in a table and a bar chart to compare model performance.
# model_types = ['Linear Regression', 'Gradiant Boosting Machine', 'Random Forest Regression']
# r2_results =  [lr_r2, GBM_r2, RFR_r2]
# mae_results = [lr_mae, GBM_mae, RFR_mae]
# mse_results = [lr_mse, GBM_mse, RFR_mse]
# df_results = pd.DataFrame({'Model Type': model_types, 'R Squared': r2_results, 'Mean Absolute Error': mae_results, 'Mean Squared Error': mse_results})
# print_table(df_results, 'model_results')
# df_results

## 7. Findings and Conclusions <a class="anchor" id="section-7"></a>

### Thoughts on Future Work and Next Steps:
-  

###  Acknowledgements:
-  This work relies on